In [1]:
import pandas as pd
import numpy as np

### 1. 데이터 로드

In [2]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [4]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])

In [5]:
### 53주차 제거

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [7]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

In [8]:
def customFunction(inputColumn):

    #inputColumn = 201601
    yearValue = inputColumn/100
    yearValue = int(yearValue)
    return yearValue

In [9]:
selloutData["YEAR"] = selloutData["YEARWEEK"].apply(customFunction)

In [10]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014


In [11]:
refinedData = selloutData[selloutData.WEEK < 53]

In [12]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2014
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2014
3,A60,PRODUCT47,201402,0.0,0.0,2,2014
4,A60,PRODUCT56,201402,23.0,23.0,2,2014


In [13]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [14]:
sortedData = refinedData.sort_values(sortKey)

In [15]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


In [16]:
step1Data = sortedData.reset_index(drop=True)

In [17]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,A00,PRODUCT34,201405,516.0,516.0,5,2014


### 1. 기본 이동평균 함수

In [18]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(
    window=15, center=True, min_periods=1).mean()

In [19]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [20]:
groupData = step1Data.groupby(groupKey)[["NEW_QTY"]].mean()

In [21]:
groupDataResult = groupData.reset_index()

In [22]:
groupDataResult.head(3)

,REGIONID,PRODUCT,YEAR,NEW_QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923


In [23]:
groupDataResult.columns=["REGIONID","PRODUCT","YEAR","MEAN_QTY"]
groupDataResult.head(2)

,REGIONID,PRODUCT,YEAR,MEAN_QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615


In [24]:
leftData = step1Data
leftData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.0


## 데이터 합치기 (속성)

In [25]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [26]:
mergedData1 = pd.merge(leftData, groupDataResult, how="left", on=joinKey)

In [27]:
len(leftData)

123864

In [28]:
len(mergedData1)

123864

In [40]:
mergedData1.to_csv("./mergedData2.csv")

In [41]:
groupKey = ["REGIONID","PRODUCT"]
groupKey2 = ["REGIONID","PRODUCT","WEEK"]

### 그룹바이함수 만들기

In [42]:
#groupDataSet = mergedData1.groupby(groupKey)

In [43]:
def groupRolling(oneGroup):

    #len(list(groupDataSet.groups))

    #oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[2])
    #oneGroup.head(1)

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5,center=True,min_periods=1).mean()

    return indexGroupData

In [44]:
finalResult = mergedData1.groupby(groupKey).apply(groupRolling)

In [45]:
finalResult2 = finalResult.groupby(groupKey2).mean()

C:\Users\SMART-15\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'REGIONID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\SMART-15\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'PRODUCT' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [47]:
finalResult2 = finalResult2.reset_index()

In [48]:
finalResult2.head()

,REGIONID,PRODUCT,WEEK,YEARWEEK,QTY,NEW_QTY,YEAR,MA,MEAN_QTY,MA2
0,A00,PRODUCT34,1,201501,277.666667,277.666667,2015,239.800000,133.057692,298.644444
1,A00,PRODUCT34,2,201502,297.333333,297.333333,2015,233.237037,133.057692,290.700000
2,A00,PRODUCT34,3,201503,236.000000,236.000000,2015,229.266667,133.057692,250.066667
3,A00,PRODUCT34,4,201504,238.000000,238.000000,2015,222.610101,133.057692,228.800000
4,A00,PRODUCT34,5,201505,201.333333,201.333333,2015,216.161111,133.057692,202.066667


In [49]:
finalResult2["SEASONALITY"] = finalResult2["NEW_QTY"] / finalResult2["MA2"]

In [50]:
finalResult3 = finalResult2.reset_index(drop=True)
finalResult3.head()

,REGIONID,PRODUCT,WEEK,YEARWEEK,QTY,NEW_QTY,YEAR,MA,MEAN_QTY,MA2,SEASONALITY
0,A00,PRODUCT34,1,201501,277.666667,277.666667,2015,239.800000,133.057692,298.644444,0.929757
1,A00,PRODUCT34,2,201502,297.333333,297.333333,2015,233.237037,133.057692,290.700000,1.022818
2,A00,PRODUCT34,3,201503,236.000000,236.000000,2015,229.266667,133.057692,250.066667,0.943748
3,A00,PRODUCT34,4,201504,238.000000,238.000000,2015,222.610101,133.057692,228.800000,1.040210
4,A00,PRODUCT34,5,201505,201.333333,201.333333,2015,216.161111,133.057692,202.066667,0.996371


In [54]:
finalResult4=finalResult3[["REGIONID","PRODUCT","WEEK","QTY","NEW_QTY","MA","MEAN_QTY","MA2","SEASONALITY"]]

In [55]:
finalResult4.head()

,REGIONID,PRODUCT,WEEK,QTY,NEW_QTY,MA,MEAN_QTY,MA2,SEASONALITY
0,A00,PRODUCT34,1,277.666667,277.666667,239.800000,133.057692,298.644444,0.929757
1,A00,PRODUCT34,2,297.333333,297.333333,233.237037,133.057692,290.700000,1.022818
2,A00,PRODUCT34,3,236.000000,236.000000,229.266667,133.057692,250.066667,0.943748
3,A00,PRODUCT34,4,238.000000,238.000000,222.610101,133.057692,228.800000,1.040210
4,A00,PRODUCT34,5,201.333333,201.333333,216.161111,133.057692,202.066667,0.996371


In [56]:
finalResult4.to_csv("./seasonality5.csv")